In [57]:
import os
import json
import pandas as pd
import traceback

In [58]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [59]:
from dotenv import load_dotenv
load_dotenv()

True

In [60]:
key=os.getenv("GOOGLE_API_KEY")

In [61]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=key,
    temperature=0.2,
)

In [62]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2


In [63]:

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [64]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [65]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text","number" ,"subject", "tone", "response_json"],
    template=TEMPLATE
)

In [66]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz" , verbose=True) 

In [67]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [68]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [69]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [70]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number","subject","tone","response_json"], output_variables=["quiz","review"],verbose=True,)

In [71]:
file_path =r"D:\Users\Lenovo\Downloads\mcqgen\data.txt"

In [72]:
file_path

'D:\\Users\\Lenovo\\Downloads\\mcqgen\\data.txt'

In [73]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [74]:
print(TEXT)

The procedure (using counting rods) for solving simultaneous linear equations now called Gaussian elimination appears in the ancient Chinese mathematical text Chapter Eight: Rectangular Arrays of The Nine Chapters on the Mathematical Art. Its use is illustrated in eighteen problems, with two to five equations.[4]

Systems of linear equations arose in Europe with the introduction in 1637 by René Descartes of coordinates in geometry. In fact, in this new geometry, now called Cartesian geometry, lines and planes are represented by linear equations, and computing their intersections amounts to solving systems of linear equations.

The first systematic methods for solving linear systems used determinants and were first considered by Leibniz in 1693. In 1750, Gabriel Cramer used them for giving explicit solutions of linear systems, now called Cramer's rule. Later, Gauss further described the method of elimination, which was initially listed as an advancement in geodesy.[5]

In 1844 Hermann G

In [75]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [76]:
NUMBER = 5
SUBJECT = "MACHINE LEARNING"
TONE = "simple"

In [77]:
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

response = generate_evaluate_chain({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON),
} , callbacks=[handler]) 



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The procedure (using counting rods) for solving simultaneous linear equations now called Gaussian elimination appears in the ancient Chinese mathematical text Chapter Eight: Rectangular Arrays of The Nine Chapters on the Mathematical Art. Its use is illustrated in eighteen problems, with two to five equations.[4]

Systems of linear equations arose in Europe with the introduction in 1637 by René Descartes of coordinates in geometry. In fact, in this new geometry, now called Cartesian geometry, lines and planes are represented by linear equations, and computing their intersections amounts to solving systems of linear equations.

The first systematic methods for solving linear systems used determinants and were first considered by Leibniz in 1693. In 1750, Gabriel Cramer used them for giving explicit solutions of linear systems, now called Cramer's rule. Later, Gauss further describ

In [78]:
import tiktoken

def estimate_tokens(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))

input_text = TEXT + json.dumps(RESPONSE_JSON) + TEMPLATE + TEMPLATE2
output_text = response["quiz"] + response["review"]
total_tokens = estimate_tokens(input_text) + estimate_tokens(output_text)

print(f"Input Tokens (estimate): {estimate_tokens(input_text)}")
print(f"Output Tokens (estimate): {estimate_tokens(output_text)}")
print(f"Total Tokens (estimate): {total_tokens}")


Input Tokens (estimate): 1079
Output Tokens (estimate): 899
Total Tokens (estimate): 1978


In [79]:
response

{'text': 'The procedure (using counting rods) for solving simultaneous linear equations now called Gaussian elimination appears in the ancient Chinese mathematical text Chapter Eight: Rectangular Arrays of The Nine Chapters on the Mathematical Art. Its use is illustrated in eighteen problems, with two to five equations.[4]\n\nSystems of linear equations arose in Europe with the introduction in 1637 by René Descartes of coordinates in geometry. In fact, in this new geometry, now called Cartesian geometry, lines and planes are represented by linear equations, and computing their intersections amounts to solving systems of linear equations.\n\nThe first systematic methods for solving linear systems used determinants and were first considered by Leibniz in 1693. In 1750, Gabriel Cramer used them for giving explicit solutions of linear systems, now called Cramer\'s rule. Later, Gauss further described the method of elimination, which was initially listed as an advancement in geodesy.[5]\n\n

In [90]:
quize=response.get("quiz")

In [ ]:
import re

def extract_json(text):
    try:
        return json.loads(text)
    except:
        try:
            json_match = re.search(r'(\{[\s\S]*\})', text)
            if json_match:
                return json.loads(json_match.group(1))
            return None
        except:
            print("Could not parse response as JSON")
            return None

quiz_json = extract_json(quize)
print(quiz_json)
if not quiz_json:
    print("Warning: Could not parse quiz as JSON. Creating manual structure.")
    quiz_json = {"1": {"mcq": "Quiz could not be parsed", "options": {"a": "-", "b": "-", "c": "-", "d": "-"}, "correct": "-"}}

{'1': {'mcq': 'The method for solving simultaneous linear equations known as Gaussian elimination was first documented in which ancient text?', 'options': {'a': "Euclid's Elements", 'b': 'Chapter Eight: Rectangular Arrays of The Nine Chapters on the Mathematical Art', 'c': "Al-Khwarizmi's The Compendious Book on Calculation by Completion and Balancing", 'd': 'The Rhind Mathematical Papyrus'}, 'correct': 'b'}, '2': {'mcq': "Who introduced the term 'matrix'?", 'options': {'a': 'Arthur Cayley', 'b': 'Hermann Grassmann', 'c': 'James Joseph Sylvester', 'd': 'Benjamin Peirce'}, 'correct': 'c'}, '3': {'mcq': 'Which mathematician is credited with introducing matrix multiplication and the inverse matrix?', 'options': {'a': 'Gauss', 'b': 'Arthur Cayley', 'c': 'Cramer', 'd': 'Descartes'}, 'correct': 'b'}, '4': {'mcq': "In what context was the term 'vector' initially introduced?", 'options': {'a': 'To represent forces in electromagnetism', 'b': 'To describe complex numbers', 'c': 'To represent a p

In [ ]:
quiz_table_data = []
for key, value in quiz_json.items():
    mcq = value.get("mcq")
    options = value.get("options")
    correct = value.get("correct")
    
    # Create a dictionary for the current row
    row = {
        "Question": mcq,
        "Option A": options.get("a"),
        "Option B": options.get("b"),
        "Option C": options.get("c"),
        "Option D": options.get("d"),
        "Correct Answer": correct
    }
    
    # Append the row to the list
    quiz_table_data.append(row)

print(quiz_table_data)

[{'Question': 'The method for solving simultaneous linear equations known as Gaussian elimination was first documented in which ancient text?', 'Option A': "Euclid's Elements", 'Option B': 'Chapter Eight: Rectangular Arrays of The Nine Chapters on the Mathematical Art', 'Option C': "Al-Khwarizmi's The Compendious Book on Calculation by Completion and Balancing", 'Option D': 'The Rhind Mathematical Papyrus', 'Correct Answer': 'b'}, {'Question': "Who introduced the term 'matrix'?", 'Option A': 'Arthur Cayley', 'Option B': 'Hermann Grassmann', 'Option C': 'James Joseph Sylvester', 'Option D': 'Benjamin Peirce', 'Correct Answer': 'c'}, {'Question': 'Which mathematician is credited with introducing matrix multiplication and the inverse matrix?', 'Option A': 'Gauss', 'Option B': 'Arthur Cayley', 'Option C': 'Cramer', 'Option D': 'Descartes', 'Correct Answer': 'b'}, {'Question': "In what context was the term 'vector' initially introduced?", 'Option A': 'To represent forces in electromagnetism

In [121]:
quiz_df = pd.DataFrame(quiz_table_data)

quiz_df.columns = ["Question", "Option A", "Option B", "Option C", "Option D", "Correct Answer"]
quiz_df.index = range(1, len(quiz_df) + 1)

styled_df = quiz_df.style.set_properties(**{
    'text-align': 'Left',
    'border': '1px solid black',
    'padding': '5px'
}).set_table_styles([{
    'selector': 'th.col0',
     
    'props': [('background-color', 'black'), ('font-weight', 'bold') ,('color', 'Green')]
}, {
     'selector': 'th',
        'props': [('background-color', 'black'), ('font-weight', 'bold') ,('color', 'blue'),('text-align', 'center')]
}])

display(styled_df)

,Question,Option A,Option B,Option C,Option D,Correct Answer
1,The method for solving simultaneous linear equations known as Gaussian elimination was first documented in which ancient text?,Euclid's Elements,Chapter Eight: Rectangular Arrays of The Nine Chapters on the Mathematical Art,Al-Khwarizmi's The Compendious Book on Calculation by Completion and Balancing,The Rhind Mathematical Papyrus,b
2,Who introduced the term 'matrix'?,Arthur Cayley,Hermann Grassmann,James Joseph Sylvester,Benjamin Peirce,c
3,Which mathematician is credited with introducing matrix multiplication and the inverse matrix?,Gauss,Arthur Cayley,Cramer,Descartes,b
4,In what context was the term 'vector' initially introduced?,To represent forces in electromagnetism,To describe complex numbers,To represent a point in space as v = xi + yj + zk,To define equipollent line segments,c
5,What development significantly boosted research into efficient algorithms for Gaussian elimination and matrix decompositions?,The invention of determinants,The development of computers,The formulation of Cramer's rule,The introduction of Cartesian geometry,b


In [122]:
quiz_df.to_csv("quiz.csv")